<a href="https://colab.research.google.com/github/mirceachira/licenta/blob/main/licenta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install backtesting
!pip install git+https://github.com/trsvchn/calabar.git

In [86]:
# Filesystem location
MOUNT_LOCATION = '/content/drive'
DATA_LOCATION = f'{MOUNT_LOCATION}/MyDrive/licenta'
TICKERS_LOCATION = f'{DATA_LOCATION}/tickers'

# Email notifications
MY_EMAIL = 'chira.mircea.darius@gmail.com'
NOTIFICATION_RECEIVER_LIST = [MY_EMAIL]

# Finance API
YAHOO_FINANCE_MAX_URL = 'https://query1.finance.yahoo.com/v7/finance/download/{TICKER_NAME}?period1=0&period2=2000000000&interval=1d&events=history&includeAdjustedClose=true'

# Tickers
TICKER_NAMES = {
    # Gaming companies
    'ATVI': 'Activision Blizzard, Inc.',
    'EA': 'Electronic Arts Inc.',
    'NTDOY': 'Nintento (traded in US)',

    # Retail
    'GME': 'GameStop Corp.',

    # Tech
    'GOOG': 'Alphabet Inc.',
}
TICKERS = list(TICKER_NAMES.keys())

In [30]:
from google.colab import drive

drive.mount(MOUNT_LOCATION)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
import pandas as pd


def get_ticker_csv_path(ticker_name):
  return f'{TICKERS_LOCATION}/{ticker_name}.csv'


def get_ticker_csv_as_df(ticker_name):
    return pd.read_csv(get_ticker_csv_path(ticker_name))

In [53]:
from calabar import Email


def send_email_notification(subject, body):
    """This will require a google app password for my gmail account
    https://myaccount.google.com/security
    """
    sender = MY_EMAIL
    receivers = NOTIFICATION_RECEIVER_LIST
    email = Email(sender, receivers)

    email.send(subject, body)

In [94]:
import requests


def download_latest_ticker_csv(ticker_name):
    yahoo_url = YAHOO_FINANCE_MAX_URL.format(TICKER_NAME=ticker_name)
    file_respone = requests.get(yahoo_url)

    csv_path = get_ticker_csv_path(ticker_name)

    print(f'Downloading to {csv_path} about {len(file_respone.content)} bytes from {yahoo_url}')
    with open(csv_path, 'wb') as f:
        f.write(file_respone.content)


def update_all_ticker_csvs():
    for ticker_name in TICKERS:
        download_latest_ticker_csv(ticker_name)


# update_all_ticker_csvs()
# download_latest_ticker_csv('GOOG')

In [91]:
from backtesting import Backtest


def test_strategy(strategy_class, ticker_name):
    stock_df = get_ticker_csv_as_df(ticker_name)
    

    print(f'Testing {strategy_class.__name__} on {ticker_name} from {stock_df["Date"].min()} to {stock_df["Date"].max()}')

    bt = Backtest(stock_df, strategy_class, commission=.002, exclusive_orders=True)
    
    stats = bt.run()

    return bt, stats


def run_through_all_tickers(strategy_class):
    results = {}

    for ticker_name in TICKERS:
        bt, stats = test_strategy(SmaCross, ticker_name)
        results[ticker_name] = stats.copy()

    return pd.concat(results, axis=1)


In [95]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


run_through_all_tickers(SmaCross)
# bt.plot()

Testing SmaCross on ATVI from 1993-10-25 to 2021-03-15
Testing SmaCross on EA from 1989-09-20 to 2021-03-15


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  # Remove the CWD from sys.path while we load stuff.


Testing SmaCross on NTDOY from 1996-11-18 to 2021-03-15
Testing SmaCross on GME from 2002-02-13 to 2021-03-15


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  # Remove the CWD from sys.path while we load stuff.


Testing SmaCross on GOOG from 2004-08-19 to 2021-03-15


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  # Remove the CWD from sys.path while we load stuff.


,ATVI,EA,NTDOY,GME,GOOG
Start,0,0,0,0,0
End,6895,7931,6119,4803,4170
Duration,6895,7931,6119,4803,4170
Exposure Time [%],99.6085,99.7226,99.5588,99.1674,98.4896
Equity Final [$],4948.35,172.568,1672.7,0,24423.6
Equity Peak [$],70813.2,10823.5,23435.8,19825.6,78943.5
Return [%],-50.5165,-98.2743,-83.273,-100,144.236
Buy & Hold Return [%],9774.67,25284.7,690.889,2090.45,4034.41
Return (Ann.) [%],NaN,NaN,NaN,NaN,NaN
Volatility (Ann.) [%],NaN,NaN,NaN,NaN,NaN
